<a href="https://colab.research.google.com/github/messi10tom/RNN-from-scratch/blob/main/RNN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
data = """During the advent of modern scientific method in the last several centuries, nature became the passive reality, organized and moved by divine laws.[3][4] With the Industrial Revolution, nature increasingly became seen as the part of reality deprived from intentional intervention: it was hence considered as sacred by some traditions (Rousseau, American transcendentalism) or a mere decorum for divine providence or human history (Hegel, Marx). However, a vitalist vision of nature, closer to the pre-Socratic one, got reborn at the same time, especially after Charles Darwin. Within the various uses of the word today, "nature" often refers to geology and wildlife. Nature can refer to the general realm of living plants and animals, and in some cases to the processes associated with inanimate objects—the way that particular types of things exist and change of their own accord, such as the weather and geology of the Earth. It is often taken to mean the "natural environment" or wilderness—wild animals, rocks, forest, and in general those things that have not been substantially altered by human intervention, or which persist despite human intervention. For example, manufactured objects and human interaction generally are not considered part of nature, unless qualified as, for example, "human nature" or "the whole of nature". This more traditional concept of natural things that can still be found today implies a distinction between the natural and the artificial, with the artificial being understood as that which has been brought into being by a human consciousness or a human mind. Depending on the particular context, the term "natural" might also be distinguished from the unnatural or the supernatural."""

In [2]:
data = ''.join([i for i in data.lower() if i.isalpha() or i in [' ', '.']])

In [3]:
import numpy as np

vocab = list(set(data))
X_train = [vocab.index(i) for i in data]
Xtr = np.zeros((len(X_train), len(vocab)))
Xtr[np.arange(len(X_train)), X_train] = 1

In [15]:
Wc = np.random.randn(100, 100 + len(vocab))
Bc = np.random.randn(100, 1)
C_tm1 = np.zeros((100, 1))
V = np.random.randn(100, 100)

W = np.random.randn(len(vocab), 100)
B = np.random.randn(len(vocab), 1)

for t in range(1, len(Xtr[:4])):
  Z_t = np.concatenate([Xtr[t-1].reshape((-1, 1)), C_tm1], axis=0)
  C_bar = Wc @ Z_t + Bc
  C = np.tanh(C_bar)
  H = np.tanh(V @ C)
  C_tm1 = C

  Y_bar = W @ H + B
  Y_hat = np.exp(Y_bar) / np.sum(np.exp(Y_bar))
  Y = Xtr[t].reshape((-1, 1))
  L = -np.sum((Y * np.log(Y_hat)))

  dy_hat = -(Y/Y_hat)
  dy_hat_b_dY_bar = []
  for i in range(len(Y_hat)):
    dy_hat_b_dY_bar.append(

        [-1 * Y_hat[i]*Y_hat[j] if i != j else (Y_hat[i]*(1 - Y_hat[i])) for j in range(len(Y_bar))]
         #  -si * sj ; i != j                        si(1 - si) ; i=j
        )
  dy_hat_b_dY_bar = np.array(dy_hat_b_dY_bar).reshape((len(Y_bar), len(Y_bar)))
  dy_bar =  dy_hat_b_dY_bar @ dy_hat
  dw = dy_bar @ H.T
  db = dy_bar


In [18]:
H.shape

(100, 1)

In [ ]:
import tensorflow as tf

a = tf.Variable(tf.random.normal((100, 100)), trainable=True)
b = tf.Variable(tf.random.normal((100, 1)), trainable=True)
with tf.GradientTape() as tape:
  c = tf.matmul(a, b)
dc_da = tape.gradient(c, a)
